# Black Box Optimization

### Data Wrangling

#### Import

###### Generalities

In [1]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
#import ast

In [2]:
# Get the parent directory of the current notebook's directory
notebook_dir = os.getcwd()
parent_dir = os.path.dirname(notebook_dir)

# Join the parent directory with "initial data"
data_path = os.path.join(parent_dir, "data")

###### Initial data

In [3]:
# import initial data

impInputInit = []

for i in range(1,9):
    pth = os.path.join(data_path, "initial_data","function_"+str(i),"initial_inputs.npy")
    impInputInit.append(np.load(pth, allow_pickle=True))

impOutputInit = []

for i in range(1,9):
    pth = os.path.join(data_path, "initial_data","function_"+str(i),"initial_outputs.npy")
    impOutputInit.append(np.load(pth, allow_pickle=True))

###### Weekly data

In [24]:
# function for transforming txt strings into np arrays

def imp_txt_to_np_array(file_path,type="input"):
    if type=="input":
        dlt="), array("
        srt=7
    elif type=="output":
        dlt="), np.float64("
        srt=12
    with open(file_path) as f:
        imp = f.read()
        imp = imp.split("]\n[")
        imp=imp[-1]
        imp = imp[srt:-2].split(dlt)
        #print(imp) if type=="output" else None

    impn = []
    for i in range(len(imp)):
        if type=="input":
            imp[i]=imp[i][1:-1].split(",")
            for j in range(len(imp[i])):
                imp[i][j]=float(imp[i][j])
            impn.append(np.array(imp[i]))
        elif type=="output":
            impn.append(float(imp[i]))
    return impn

In [52]:
# import new weekly data

week_now = 3

impInputNew=[]


for j in range(1, week_now +1):
    pth=os.path.join(data_path,"incremental_data", "week_"+str(j),"inputs.txt")
    impInputNew.append(imp_txt_to_np_array(pth))


new=[[] for _ in range(len(impInputNew[0]))]

for i in range(len(impInputNew[0])):
    for n in range(week_now):
        new[i].append(impInputNew[n][i])

impInputIncr=[None for i in range(8)]


for i in range(len(new)):
    #print(len(impInputInit[i])) #check
    impInputIncr[i]=impInputInit[i]

for nw_idx, nw in enumerate(impInputNew):
    for i in range(len(new)):
        impInputIncrn = impInputIncr[i]
        impInputIncrn = np.append( impInputIncrn,[np.asarray(new[i][nw_idx])] , axis=0)

        impInputIncr[i] = impInputIncrn

#check
#print("\n")
#for i in range(len(new)):
#    print(len(impInputIncr[i]))
#    print(len(impInputIncr[i][-1]))



impOutputNew=[]


for j in range(1, week_now +1):
    pth=os.path.join(data_path,"incremental_data", "week_"+str(j),"outputs.txt")
    impOutputNew.append(imp_txt_to_np_array(pth,type="output"))


new=[[] for _ in range(len(impOutputNew[0]))]

for i in range(len(impOutputNew[0])):
    for n in range(week_now):
        new[i].append(impOutputNew[n][i])

impOutputIncr=[None for i in range(8)]


for i in range(len(new)):
    #print(len(impOutputInit[i])) #check
    impOutputIncr[i]=impOutputInit[i]

for nw_idx, nw in enumerate(impOutputNew):
    for i in range(len(new)):
        impOutputIncrn = impOutputIncr[i]
        impOutputIncrn = np.append( impOutputIncrn,[np.asarray(new[i][nw_idx])] , axis=0)

        impOutputIncr[i] = impOutputIncrn

#check
#print("\n")
#for i in range(len(new)):
#    print(len(impOutputIncr[i]))
#    print(len(impOutputIncr[i][0]))


#### Definition

In [74]:
#data

data=[]

for i in range(len(impInputIncr)):
    data.append(list(zip(impInputIncr[i], impOutputIncr[i])))

print(data[1])

#print(list(map(lambda x: x[0], data[0])))

dimensions={1: 2, 2: 2, 3: 3, 4: 4, 5: 4, 6: 5, 7: 6, 8: 8}

positiveQ={1: False, 2: True, 3: False, 4: False, 5: True, 6: True, 7: True, 8: True}


[(array([0.66579958, 0.12396913]), 0.5389961189269181), (array([0.87779099, 0.7786275 ]), 0.42058623962798264), (array([0.14269907, 0.34900513]), -0.06562362443733738), (array([0.84527543, 0.71112027]), 0.293992912410866), (array([0.45464714, 0.29045518]), 0.2149645101004509), (array([0.57771284, 0.77197318]), 0.023105549798190586), (array([0.43816606, 0.68501826]), 0.24461934400448035), (array([0.34174959, 0.02869772]), 0.0387490151561584), (array([0.33864816, 0.21386725]), -0.013857618149729824), (array([0.70263656, 0.9265642 ]), 0.6112052157614438), (array([0.733333, 0.933333]), 0.568832914947192), (array([0.8     , 0.133333]), 0.07154178784077911), (array([0.6, 0. ]), 0.08943228274023461)]


In [ ]:
#grid

def gridAll(k, sub=5):
    dim = dimensions[k]
    zero = np.zeros(dim) * sub
    one = np.ones(dim) * sub
    
    # Create a mesh grid within the space defined by (zero, one)
    grid_points = np.meshgrid(*[np.linspace(0, 1, sub+1) for _ in range(dim)])
    
    # Flatten the grid points and combine into a single array of coordinates
    grid_points_flat = np.array(grid_points).reshape(dim, -1).T
    return grid_points_flat

## Training

### Gaussian Process Regressor

In [53]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF


In [75]:
def create_predictor(X, y):
    kernel = RBF()  # Radial basis function kernel
    gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)
    gp.fit(X, y)
    return gp


In [76]:
predictors=[]

for k in range(8):
    #print(data[k])
    X = list(map(lambda x: x[0], data[k]))
    y = list(map(lambda x: x[1], data[k]))
    #print(X)
    #print(y)
    predictors.append(create_predictor(X, y))


X_test = np.array([0.4,0.9])
y_pred, sigma = predictors[1].predict([X_test], return_std=True)

print(f"Predicted value: {y_pred[0]}, Standard Deviation: {sigma[0]}")

Predicted value: 0.7098260658140264, Standard Deviation: 0.6242483699968204


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/gaussian_process/_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
